In [113]:
from pyscf import gto,scf
import numpy as np 
from numpy.linalg import norm as norm
from scipy.spatial.transform import Rotation as R
import matplotlib.pyplot as plt
from pyscf.grad import rhf as grhf
from pyscf.hessian import rhf as hrhf
import basis_set_exchange as bse
import copy
from berny import Berny, geomlib,Geometry,Math
import berny.coords
import sys
sys.path.append("../../Libs/")
from aaff import aaff,alc_deriv_grad_nuc,alc_differential_grad_nuc
from FcMole import FcM,FcM_like
from alch_deriv_NEW import alch_deriv
from Morse import mpa, Morse_interpolator
from AP_class import APDFT_perturbator as AP
from FDcoeffs import *
from ABSE import absec
ang2bohr=1.8897261246
bohr2ang=.5291772109
%load_ext autoreload
%autoreload 2
from AP_class import APDFT_perturbator as AP

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [114]:
%load_ext autoreload
%autoreload 2
from AP_class import APDFT_perturbator as AP

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [115]:
def nr_step(ic,g,h):
    ncoords=ic.shape[0]*ic.shape[1]
    p1cds=ic-np.linalg.solve((np.swapaxes(h,1,2)).reshape(ncoords,ncoords),g.reshape(-1)).reshape(ic.shape[0],ic.shape[1])
    p1cds-=p1cds[0]
    print (norm(p1cds[1]))
    if p1cds.shape[0]>2:
        print (anglecenter(p1cds),anglecenter(p1cds)*np.pi/180)
    return(p1cds)
def anglecenter(coords):
    return np.around(np.arccos(coords[1].dot(coords[2])/norm(coords[1])/norm(coords[2]))*180/np.pi,decimals=3)

In [162]:
def build_h_ic(s,g_ic,h0,B_inv):
    geom0=s.geom.copy()
    B=s.coords.B_matrix(geom0)
    bms=[]
    for i in range(geom0.coords.flatten().shape[0]):
        a=geom0.coords.flatten()
        geom1=geom0.copy()
        a[i]+=.001*bohr2ang
        a=a.reshape(geom0.coords.shape)
        geom1.coords=a
        bms.append((s.coords.B_matrix(geom1)-B)*1000)
    bms_arr=np.asarray(bms)
    BpG2=np.einsum('ijk,j->ik',bms,g_ic)
    h_ic=B_inv.T@(h0-BpG2)@B_inv
    return h_ic

ebde=100/627.5 # empirical bond dissociation energy
def mpa_pb_with_energy(coords,atoml,g,h,e,ghost=[]):
    g=np.asarray(g)
    if not len(h.shape)==2:
        h=h.swapaxes(1,2)
        h=h.reshape(g.shape[0]*3,g.shape[0]*3)
    geom0=Geometry(atoml,coords*bohr2ang)
    bernyobj=Berny(geom0)
    s=bernyobj._state
    B = s.coords.B_matrix(geom0)
    q0=s.coords.eval_geom(geom0)
    B_inv = B.T.dot(Math.pinv(np.dot(B, B.T)))
    g_ic=np.dot(B_inv.T, (g).reshape(-1))

    h_ic=build_h_ic(s,g_ic,h,B_inv)
    bnr=0
    dde_mb=0
    ddq_mb=np.zeros_like(q0)
    for i in range(len(s.coords._coords)):
        if s.coords._coords[i].__class__ is berny.coords.Bond:
            bnr+=1
            if s.coords._coords[i].i not in ghost and s.coords._coords[i].j not in ghost:
                MorseInt=Morse_interpolator(g_ic[i],h_ic[i,i],q0[i],ebde,e)
                ddq_mb[i]+=MorseInt.re+g_ic[i]/h_ic[i,i]-q0[i]
                dde_mb+=MorseInt.e_min-e+1/2*h_ic[i,i]*(MorseInt.re-q0[i])**2
    dq_NR=-np.linalg.solve(h_ic,g_ic)
    ddq_mb[bnr:]=np.linalg.solve(h_ic[bnr:,bnr:],-h_ic[bnr:,:]@(ddq_mb))
    dq_tot=dq_NR+ddq_mb
    en_MB=(e-1/2*np.einsum("i,ij,j",dq_tot,h_ic,dq_tot)+dde_mb)
    # only for deprot
    q0[g.shape[0]-1:]*=180/np.pi
    dq_tot[g.shape[0]-1:]*=180/np.pi
    return q0,q0+dq_tot,en_MB
    

In [157]:
optgs=np.load("optgeoms.npy",allow_pickle=True)
ch4optg,nh3optg,oh2optg,hfoptg=np.load("optgeoms.npy",allow_pickle=True)
for i in [ch4optg,nh3optg,oh2optg]:
    print(anglecenter(i),norm(i[1]))
ch4str=("C {} {} {}"+"; H {} {} {}"*4)
ch4str=ch4str.format(*ch4optg.flatten().tolist())
nh3str=("N {} {} {}"+"; H {} {} {}"*3)
nh3str=nh3str.format(*nh3optg.flatten().tolist())
oh2str=("O {} {} {}"+"; H {} {} {}"*2)
oh2str=oh2str.format(*oh2optg.flatten().tolist())
hfstr=("F {} {} {} ; H {} {} {}")
hfstr=hfstr.format(*hfoptg.flatten().tolist())

109.471 2.043855535391837
108.214 1.8855204253782516
106.398 1.775591054614095


In [149]:
for i in ch4optg:
    print(norm(i))

0.0
2.043855535391837
2.0438571177507976
2.0438571177510663
2.0438534958418515


In [150]:
pcxbs={"H":"pc-2",'C':bse.get_basis("pcX-2",fmt="nwchem",elements=[6]),'N':bse.get_basis("pcX-2",fmt="nwchem",elements=[7])\
      ,'O':bse.get_basis("pcX-2",fmt="nwchem",elements=[8]),'F':bse.get_basis("pcX-2",fmt="nwchem",elements=[9])}

In [151]:
CH4=gto.M(atom=ch4str,unit="Bohr",basis=pcxbs,verbose=0)
NH3=gto.M(atom=nh3str,unit="Bohr",basis=pcxbs,verbose=0)
H2O=gto.M(atom=oh2str,unit="Bohr",basis=pcxbs,verbose=0)
HF=gto.M(atom=hfstr,unit="Bohr",basis=pcxbs,verbose=0)

In [156]:
CH4scf=scf.RHF(CH4)
NH3scf=scf.RHF(NH3)
H2Oscf=scf.RHF(H2O)
HFscf=scf.RHF(HF)
[CH4scf.scf(),NH3scf.scf(),H2Oscf.scf(),HFscf.scf()]

[-40.21518040349943,
 -56.22218363575204,
 -76.06331933366663,
 -100.06554016873311]

In [161]:
[norm(i[1]) for i in optgs],[anglecenter(i) for i in optgs[:-1]]

([2.043855535391837,
  1.8855204253782516,
  1.775591054614095,
  1.6957388734575747],
 [109.471, 108.214, 106.398])

In [153]:
phf=scf.RHF(FcM_like(NH3,fcs=[[0,NH3.natm-1],[1,-1]]))
phf.scf(dm0=phf.init_guess_by_1e())

-76.0566737693386

In [154]:
def dzlist(n):
    l=np.zeros(n)
    l[0]=1
    l[-1]=-1
    return l.tolist()
aaff(NH3scf,dzlist(NH3.natm))

array([[-6.62394403e-01, -1.59408708e+00,  8.70661696e-01],
       [ 2.60037227e-01, -7.00695605e-02,  1.22957382e-03],
       [-9.47212099e-02,  1.75514410e-01,  1.80977849e-01],
       [ 4.97078387e-01,  1.48864223e+00, -1.05286912e+00]])

In [155]:
apnh3=AP(NH3scf,sites=[0,3])
apnh3.build_all()

In [125]:
apnh3.APDFT3([1,-1])

-76.04751220000706

In [126]:
apnh3.third_deriv([1,-1])

0.7036652791070789

In [127]:
def e_g_h_predict(mol):
    ds=mol.natm-1
    hf=scf.RHF(mol)
    hfp=scf.RHF(FcM_like(mol,fcs=[[0,ds],[.1,-.1]]))
    hfm=scf.RHF(FcM_like(mol,fcs=[[0,ds],[-.1,.1]]))
    hf.scf()
    hfp.scf(dm0=hfp.init_guess_by_1e())
    hfm.scf(dm0=hfm.init_guess_by_1e())
    # energy prediction 
    ap0=AP(hf,sites=[0,ds])
    ap0.build_all()
    app=AP(hfp,sites=[0,ds])
    app.build_all()    
    apm=AP(hfm,sites=[0,ds])
    apm.build_all()
    E_p=ap0.APDFT3([1,-1])+(app.third_deriv([1,-1])-apm.third_deriv([1,-1]))/.2/24 \
                +(app.third_deriv([1,-1])+apm.third_deriv([1,-1])-2*ap0.third_deriv([1,-1])) /.01/120
    
    #gradient prediction
    g0=hf.Gradients().grad()
    af0=aaff(hf,dzlist(mol.natm))
    afp=aaff(hfp,dzlist(mol.natm))
    afm=aaff(hfm,dzlist(mol.natm))
    G_p=g0+af0+(afp-afm)/.2/2+(afp+afm-2*af0)/.01/6+alc_differential_grad_nuc(mol, dzlist(mol.natm))
    G_p[0]+=G_p[-1]  # clearing ghost atom component
    G_p=G_p[:-1]
    
    #Hessian prediction
    H0=hf.Hessian().hess()
    Hp=hfp.Hessian().hess()
    Hm=hfm.Hessian().hess()
    H_pred=H0+(Hp-Hm)/.2+(Hp+Hm-2*H0)/.01/2
    H_pred=H_pred[:-1,:-1,:,:]
    return mol.atom_coords()[:-1],[mol.atom_pure_symbol(x) for x in range(ds)] ,G_p,H_pred,E_p

In [128]:
[NH3.atom_pure_symbol(x) for x in range(NH3.natm)]

['N', 'H', 'H', 'H']

In [163]:
mpa_pb_with_energy(*e_g_h_predict(CH4))

(array([  2.04385554,   2.04385712,   2.04385712, 109.47124618,
        109.47124618, 109.47111993]),
 array([  1.87851895,   1.87852176,   1.87852775, 109.90195441,
        109.90195652, 109.90184832]),
 -56.22772108399102)

In [141]:
-56.227721083995064+absec("C","N")

-56.228208079204336

In [164]:
mpa_pb_with_energy(*e_g_h_predict(NH3))

(array([  1.88552043,   1.8855208 , 108.21427696]),
 array([  1.77743108,   1.7774306 , 107.04721616]),
 -76.06948141614552)

In [140]:
-76.06948141614544+absec("N","O")

-76.07004886303703

In [143]:
mpa_pb_with_energy(*e_g_h_predict(H2O))

(array([1.77559105]), array([1.69304669]), -100.07191544872327)

In [138]:
-100.07191544872308+absec("O","F")

-100.07271455687518